In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import shap
import joblib
import matplotlib.pyplot as plt

# Load cleaned data
df = pd.read_csv('cleaned_layoffs.csv')

# Target column (assuming we define a binary outcome)
df['layoff_flag'] = (df['percentage_laid_off'] > 0.5).astype(int)

# Feature engineering
features = ['company', 'industry', 'country', 'stage', 'funds_raised_usd']
df = df[features + ['layoff_flag']].dropna()

# Encode categoricals
le = LabelEncoder()
for col in ['company', 'industry', 'country', 'stage']:
    df[col] = le.fit_transform(df[col])

X = df.drop('layoff_flag', axis=1)
y = df['layoff_flag']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# SHAP interpretation
explainer = shap.Explainer(model)
shap_values = explainer(X_test)
shap.summary_plot(shap_values, X_test)

# Save model
joblib.dump(model, '../scripts/xgb_layoff_model.pkl')
print("Model saved.")